In [1]:
#retriever and Chain

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("GenAI.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'GenAI.pdf', 'page': 0}, page_content='Generative AI: Concepts and Applications\n1. Introduction\nGenerative AI refers to a class of artificial intelligence algorithms that generate new content, such as\ntext, images, or audio, by learning from large datasets. The key difference between generative AI\nand traditional AI models is that generative models can create new data, whereas traditional models\ntypically only analyze and interpret data.\n2. Key Concepts\n- Natural Language Generation: Creating human-like text based on input prompts.\n- Image Synthesis: Generating new images using models like GANs or diffusion models.\n- Generative Models: Including architectures like Generative Adversarial Networks (GANs),\nvariational autoencoders (VAEs), and transformer models like GPT.\n3. Popular Generative Models\n- GANs (Generative Adversarial Networks): GANs consist of two neural networks, a generator and a\ndiscriminator, that compete with each other to creat

In [2]:
import os   # call environment variables
from dotenv import load_dotenv

load_dotenv()

# Langsmith
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
# recursive character splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunk_document = text_splitter.split_documents(docs)
chunk_document

[Document(metadata={'source': 'GenAI.pdf', 'page': 0}, page_content='Generative AI: Concepts and Applications\n1. Introduction\nGenerative AI refers to a class of artificial intelligence algorithms that generate new content, such as\ntext, images, or audio, by learning from large datasets. The key difference between generative AI\nand traditional AI models is that generative models can create new data, whereas traditional models\ntypically only analyze and interpret data.\n2. Key Concepts\n- Natural Language Generation: Creating human-like text based on input prompts.'),
 Document(metadata={'source': 'GenAI.pdf', 'page': 0}, page_content='2. Key Concepts\n- Natural Language Generation: Creating human-like text based on input prompts.\n- Image Synthesis: Generating new images using models like GANs or diffusion models.\n- Generative Models: Including architectures like Generative Adversarial Networks (GANs),\nvariational autoencoders (VAEs), and transformer models like GPT.\n3. Popular 

In [4]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
embedding = OllamaEmbeddings(
    model = "llama2",
)

db = FAISS.from_documents(chunk_document, embedding = embedding)
db

In [5]:
query = "Describe the Key concepts of Generative AI"
retrieved_result = db.similarity_search(query)
print(retrieved_result[0].page_content)

- GPT (Generative Pre-trained Transformer): The GPT family of models, such as GPT-3, is designed
to generate human-like text by predicting the next word in a sequence based on large-scale training
data.
4. Applications of Generative AI
- Text generation for chatbots and virtual assistants.
- Image generation for art and design.
- Audio generation for voice assistants and music creation.
- Data augmentation for training machine learning models.
5. Conclusion


In [6]:
# Design Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $10 if the user finds the answer helpful.
<context>
{context}
</context>

Question: {input}                                        
                                          
""")


In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model ="llama2")
llm


Ollama()

In [8]:
# Chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context.\nThink step by step before providing a detailed answer.\nI will tip you $10 if the user finds the answer helpful.\n<context>\n{context}\n</context>\n\nQuestion: {input}                                        \n                                          \n'), additional_kwargs={})])
| Ollama()
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [9]:
# Retrievers
 # It craetes a interface and take data from vector store and display it
retriever = db.as_retriever() 
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000149EEEC1970>, search_kwargs={})

In [10]:

"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/ 
"""

from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document_chain)


In [11]:
response =retriever_chain.invoke({"input": "What is Generative AI"})
response['answer']

'Generative AI refers to a class of artificial intelligence algorithms that generate new content, such as text, images, or audio, by learning from large datasets. These models are designed to create new data that resembles the original training data, rather than simply analyzing and interpreting it like traditional machine learning models. Generative AI models are trained on vast amounts of data and use complex neural networks to learn patterns and relationships within the data. They can then generate new content that is similar but not identical to the original training data.\n\nThe key difference between generative AI and traditional AI models is that generative models can create new data, whereas traditional models typically only analyze and interpret data. Generative AI has numerous applications across various industries, including text generation for chatbots and virtual assistants, image generation for art and design, audio generation for voice assistants and music creation, and 